In [23]:
#Import Libraries
import cv2
from ultralytics import YOLO
import tkinter as tk
from tkinter import Label, Button
from PIL import Image, ImageTk

In [24]:
#Filepath of the trained model
model_path = 'trained_models/yolov5_screwtype_trained.pt'

# Load the trained model for real-time detection
model = YOLO(model_path)

# Get class names from the model
class_names = model.names

In [25]:
# Initialize Tkinter window
root = tk.Tk()
root.title("Screw Real-Time Component Detection")

# Create label for video feed
video_label = Label(root)
video_label.pack()

# Create label for detection details
details_label = Label(root, text="Detected Objects: ", font=("Arial", 12), justify=tk.LEFT)
details_label.pack()

In [26]:
# Open webcam (but don't start capturing yet)
cap = None 
running = False # Flag to control detection loop 

def start_detection():
    global cap, running 
    if not running:
        cap = cv2.VideoCapture(1) # Open webcam
        running = True # Set up video capture
        video_label.config(text="Starting detection...") 
        detect_objects()

def stop_detection():
    global cap, running
    if running:
        running = False # Stop detection loop
        cap.release() # Release webcam
        cv2.destroyAllWindows() # Close all OpenCV windows
        video_label.config(image='') # Clear video label
        details_label.config(text="Detection Stopped") 

def detect_objects():
    global running
    if not running:
        return
    
    ret, frame = cap.read()
    if not ret:
        return
    
    # Run object detection
    results = model(frame) # Perform detection on the frame
    detected_text = "Detected Objects:\n"
    
    for result in results:
        for box in result.boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0]) # Get bounding box coordinates
            conf = box.conf[0].item() 
            cls = int(box.cls[0].item()) 
            class_name = class_names.get(cls, f"Class {cls}") 
            label = f"{class_name}: {conf:.2f}" 
            detected_text += label + "\n" 
            
            # Draw bounding box
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2) 
            cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2) 
    
    # Convert frame to Tkinter format
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB) # Convert BGR to RGB
    frame = Image.fromarray(frame) # Convert to PIL Image
    frame.thumbnail((600, 400)) # Resize for display
    imgtk = ImageTk.PhotoImage(frame) 
    
    # Update video feed
    video_label.config(image=imgtk) 
    video_label.image = imgtk # Keep a reference to avoid garbage collection
    
    # Update detection details
    details_label.config(text=detected_text) 
    
    video_label.after(10, detect_objects)  # Refresh frame every 10 ms

In [27]:
# Create start and stop buttons
start_button = Button(root, text="Start Detection", command=start_detection, font=("Arial", 12)) 
start_button.pack(pady=5) 

stop_button = Button(root, text="Stop Detection", command=stop_detection, font=("Arial", 12))
stop_button.pack(pady=5)

# Start Tkinter loop
root.mainloop()



0: 480x640 1 Wood screw, 53.3ms
Speed: 2.3ms preprocess, 53.3ms inference, 11.3ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 Wood screw, 55.9ms
Speed: 3.2ms preprocess, 55.9ms inference, 10.3ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 Wood screw, 52.7ms
Speed: 2.5ms preprocess, 52.7ms inference, 8.9ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 Wood screw, 43.1ms
Speed: 3.0ms preprocess, 43.1ms inference, 5.3ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 Wood screw, 39.7ms
Speed: 3.6ms preprocess, 39.7ms inference, 3.2ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 Wood screw, 22.2ms
Speed: 2.0ms preprocess, 22.2ms inference, 3.9ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 29.1ms
Speed: 2.4ms preprocess, 29.1ms inference, 1.1ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 Wood screw, 23.1ms
Speed: 2.5ms preprocess, 23.1ms inference, 3.9ms pos

Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\Users\gauta\AppData\Local\Programs\Python\Python312\Lib\tkinter\__init__.py", line 1968, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "C:\Users\gauta\AppData\Local\Temp\ipykernel_7392\4240056366.py", line 18, in stop_detection
    cv2.destroyAllWindows() # Close all OpenCV windows
    ^^^^^^^^^^^^^^^^^^^^^^^
cv2.error: OpenCV(4.11.0) D:\a\opencv-python\opencv-python\opencv\modules\highgui\src\window.cpp:1295: error: (-2:Unspecified error) The function is not implemented. Rebuild the library with Windows, GTK+ 2.x or Cocoa support. If you are on Ubuntu or Debian, install libgtk2.0-dev and pkg-config, then re-run cmake or configure script in function 'cvDestroyAllWindows'

